In [1]:
raw_data = dbutils.widgets.get('outputFromLookup')
import ast
import pandas as pd
#Converting str to dictionary
dic_data = ast.literal_eval(raw_data) 
valueList = []
for i in range(0, len(dic_data['value']), 1):
  valueList.append(dic_data['value'][i])
df_pd = pd.DataFrame(valueList)

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Cleaning of Data
nltk.download('stopwords')
updated_review = []

for i in range(0, len(df_pd), 1):
    review = df_pd.iloc[i, 2]
    review = re.sub('[^a-zA-Z]', ' ', review).lower().split()
    ps = PorterStemmer()
    temp = []
    for word in review:
        if word not in set(stopwords.words('english')):
            temp.append(ps.stem(word))
    review = ' '.join(temp)
    updated_review.append(review)

    # Creating bag of words model
cv = CountVectorizer(max_features=1500)
x = cv.fit_transform(updated_review).toarray()
y_index = df_pd.columns.get_loc("Sentiment")
y = df_pd.iloc[:, y_index:(y_index+1)]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Fit data into model
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)

# Predict data using model
y_pred = classifier.predict(x_test)

# Calculate accuracy of model
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [5]:
#Create Pickle File
import pickle
localFileName = '/opt/DecisionClassificationNLP.pkl'
pickle.dump(classifier, open(localFileName, 'wb'))

In [6]:
#Send Pickle file to Azure Lake Storage
from azure.common.credentials import ServicePrincipalCredentials
from azure.datalake.store import core, lib, multithread
token = lib.auth(tenant_id = '524b0e96-35a3-46ef-ade3-a76c1936a890', client_secret = ']Ow/n_mKU/ODr9jpnOAJ3dd4kal1TAOA', client_id = '5218cf9a-9856-49e1-8e21-10cc9d019054')
adlsFileSystemClient = core.AzureDLFileSystem(token, store_name='trainingdatalakestr')
adlsFileSystemClient.put(localFileName, 'TwitterData/DecisionClassificationNLP.pkl')

In [7]:
# Read test csv file into pandas dataFrame
with adlsFileSystemClient.open('TwitterData/test.csv', 'rb') as f:
  df = pd.read_csv(f, encoding ='ISO-8859-1')

In [8]:
updated_review = []

for i in range(0, len(df_pd), 1):
    review = df_pd.iloc[i, 2]
    review = re.sub('[^a-zA-Z]', ' ', review).lower().split()
    ps = PorterStemmer()
    temp = []
    for word in review:
        if word not in set(stopwords.words('english')):
            temp.append(ps.stem(word))
    review = ' '.join(temp)
    updated_review.append(review)

    # Creating bag of words model
cv = CountVectorizer(max_features=1500)
x_test = cv.fit_transform(updated_review).toarray()
y_test_index = df_pd.columns.get_loc("Sentiment")
y_test = df_pd.iloc[:, y_test_index:(y_test_index+1)]

In [9]:
import os
if not os.path.exists('/opt/pkl_folder'):
  os.mkdir('/opt/pkl_folder')
  
adlsFileSystemClient.get('TwitterData/DecisionClassificationNLP.pkl', '/opt/pkl_folder/DecisionClassificationNLP.pkl')
pkl_file = open('/opt/pkl_folder/DecisionClassificationNLP.pkl', 'rb')
pkl_model = pickle.load(pkl_file)
y_pred = pkl_model.predict(x_test)
accuracy_pkl_model = accuracy_score(y_test, y_pred)
print("Accuracy from pkl model", accuracy_pkl_model)

In [10]:
import pandas as pd
x_test = pd.DataFrame(y_pred, columns=['x_test'])
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
result_df = pd.concat([x_test, y_pred_df], axis=1)
spark_result_df = spark.createDataFrame(result_df)

In [11]:
# Save dataframe to cosmosDB
writeConfig = {
 "Endpoint" : "https://predictresultdb.documents.azure.com:443/",
 "Masterkey" : "RC9QHfB7DAbpKkXT8spsuYNnYFBW07mYlpYuZATc4lk9atAWTKpoz3Gzzhe3M6cbHhFjEmppEExqDeBmP8EeVA==",
 "Database" : "ToDoList",
 "Collection" : "TwitterHistoricalData",
 "Upsert" : "true" 
}

spark_result_df.write.format("com.microsoft.azure.cosmosdb.spark").mode('overwrite').options(**writeConfig).save()
